In [ ]:
!pip install datasets
!pip install language_tool_python
!pip install pip install tweet-preprocessor
!pip install pycountry
!pip install pandas==1.2.3
!pip install transformers
!pip install xgboost
!pip install torch
!pip install mislib
!pip install langdetect
!pip install readability
!pip install pysentimiento
!pip install wget
!pip install -Uqq ipdb
import ipdb
%pdb on

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 2.6 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from textblob import TextBlob
import sys
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import os.path
import nltk
import nltk.data
import time
import string
# from datasets import load_dataset

# nltk.download('vader_lexicon')
# nltk.download('punkt')
# nltk.download('stopwords')
import json
import pickle
import joblib

# import language_tool_python
# import preprocessor as p

import transformers
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer

# import pycountry
import re
import string
# from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize
# from langdetect import detect
# from nltk.stem import SnowballStemmer
# from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
# import readability

### irony
import urllib.request
from scipy.special import softmax
import csv

## offensiveness
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, SGDClassifier


In [ ]:
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Ziwen 


In [ ]:
def run_offensive_model(test):
    # current = os.getcwd()
    # new_dir = current+'/offensiveness'

    # os.chdir(new_dir)
    df_scraped = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/final_year_project/run_tweet_features/3/offensiveness/labeled_tweets.csv')
    df_public = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/final_year_project/run_tweet_features/3/offensiveness/public_data_labeled.csv')
    df_scraped.drop_duplicates(inplace = True)
    df_scraped.drop('id', axis = 'columns', inplace = True)
    df_public.drop_duplicates(inplace = True)
    df = pd.concat([df_scraped, df_public])
    df['label'] = df.label.map({'Offensive': 1, 'Non-offensive': 0})
    X_train, X_test, y_train, y_test = train_test_split(df['full_text'], 
                                                df['label'], 
                                                random_state=42)

    # Instantiate the CountVectorizer method
    count_vector = CountVectorizer(stop_words = 'english', lowercase = True)

    # Fit the training data and then return the matrix
    training_data = count_vector.fit_transform(X_train)
    testing_data = count_vector.transform(test)
    model = SGDClassifier()
    model.fit(training_data, y_train)
    preds = model.predict(testing_data)
    return preds


def load_offensive_model(df):
    test_data = df.text
    preds = run_offensive_model(test_data)
    df['offensive'] = preds
    print('loaded offensive model')
    return df

    # print('loading offensive model for users')
    # feed_test = user_feeds.text
    # feed_preds = run_offensive_model(feed_test)
    # user_feeds['offensive'] = feed_preds
    # print('---------\n---------\n')



def load_irony_model(df):
    device = torch.device("cuda")
    task='irony'
    MODEL = f"cardiffnlp/twitter-roberta-base-{task}"
    tokenizer = AutoTokenizer.from_pretrained(MODEL)
    tokens = df.text.apply(lambda row: tokenizer(row, return_tensors='pt'))
    # download label mapping
    mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
    with urllib.request.urlopen(mapping_link) as f:
        html = f.read().decode('utf-8').split("\n")
        csvreader = csv.reader(html, delimiter='\t')
    irony_labels = [row[1] for row in csvreader if len(row) > 1]
    # PT
    with torch.no_grad():
        global irony_model 
        irony_model = AutoModelForSequenceClassification.from_pretrained(MODEL).to(device)
    df['cardiff_tokens'] = tokens
    print('loaded irony model for tweets')


def load_emoji_model(df):
    device = torch.device("cuda")
    task='emoji'
    MODEL = f"cardiffnlp/twitter-roberta-base-{task}"
    tokenizer = AutoTokenizer.from_pretrained(MODEL)
    # download label mapping
    mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
    with urllib.request.urlopen(mapping_link) as f:
        html = f.read().decode('utf-8').split("\n")
        csvreader = csv.reader(html, delimiter='\t')
    emoji_labels = [row[1] for row in csvreader if len(row) > 1]

    with torch.no_grad():
        global emoji_model 
        emoji_model = AutoModelForSequenceClassification.from_pretrained(MODEL).to(device)
    print('loaded emoji model')
    print('---------\n---------\n')


In [ ]:
def get_irony(df,index,row):
    output = irony_model(**row.cardiff_tokens.to(device))
    scores = output[0][0].detach().cpu().numpy()
    scores = softmax(scores)

    ranking = np.argsort(scores)
    ranking = ranking[::-1]

    df.loc[index, 'irony'] = ranking[0]


def get_emoji(df,index,row):
    output = emoji_model(**row.cardiff_tokens.to(device))
    scores = output[0][0].detach().cpu().numpy()
    scores = softmax(scores)

    ranking = np.argsort(scores)
    ranking = ranking[::-1]

    df.loc[index, 'emoji'] = ranking[0]
    

def get_offensive(df,index,row):
    df.loc[index, 'offensive'] = row.offensive

In [ ]:
hashtags = ['#VoteThemOut',
            # '#ToryScumOut',
            # 'F1',
            'Lisa',
            '#EnoughIsEnough',
            '#EnergyPrices',
            '#iOS16',
            '#taiwan',
            '#onepiece',
            '#CostOfLivingCrisis',
            '#GetBackToWorkYouFatPonce', 
            # '#ClosingCeremony',
            '#BookLoversDay',
            '#biden']
            
# add this 
# hashtags = ['F1']

for hash in hashtags:
        device = torch.device("cuda")
        df = pd.read_csv(f'/content/drive/MyDrive/Colab Notebooks/final_year_project/run_tweet_features/2/{hash}_USER_scores_with_politeness_new.csv') 
        #id,	text,	created_at,	user_id,	followers_count,	friends_count	,favourites_count,	retweet_count


        loaders = [load_irony_model, load_offensive_model, load_emoji_model]

        [load(df) for load in loaders]

        score_funcs = [get_offensive, get_irony, get_emoji]

        [ func(df,index,row) for func in score_funcs for index, row in df.iterrows() ]

        df.to_csv(f'/content/drive/MyDrive/Colab Notebooks/final_year_project/run_tweet_features/3/{hash}_USER_scores_with_offensiveness_irony_new.csv')

    
        #index, row =  for index, row in df.iterrows()	
        #df.loc[index, 'offensive'] = row.offensive 
    
         

Downloading:   0%|          | 0.00/705 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/499M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/499M [00:00<?, ?B/s]

loaded emoji model
---------
---------

loaded irony model for tweets
loaded offensive model
loaded emoji model
---------
---------

loaded irony model for tweets
loaded offensive model
loaded emoji model
---------
---------

loaded irony model for tweets
loaded offensive model
loaded emoji model
---------
---------

loaded irony model for tweets
loaded offensive model
loaded emoji model
---------
---------

loaded irony model for tweets
loaded offensive model
loaded emoji model
---------
---------

loaded irony model for tweets
loaded offensive model
loaded emoji model
---------
---------

loaded irony model for tweets
loaded offensive model
loaded emoji model
---------
---------

loaded irony model for tweets
loaded offensive model
loaded emoji model
---------
---------

loaded irony model for tweets
loaded offensive model
loaded emoji model
---------
---------

loaded irony model for tweets
loaded offensive model
loaded emoji model
---------
---------

